In [227]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pickle
from glob import glob
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import random_split
import pandas as pd
import time
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyper-parameter

In [228]:
batch_size = 256
in_dim = 4
in_len = 19
out_dim = 4
out_len = 30
hidden_dim = 128
num_layers = 2
learning_rate = 0.001
decay_rate = 0.95
num_epoch = 1000
name = 'cnn_1_32'
model_name = name + '_model.pth'
submission_name = name + '_submission.csv'

# CNN

In [229]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        self.conv1 = nn.Conv1d(4, 32, kernel_size=5)
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(480, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 60)
        )
        
    def forward(self, x):
        out = nn.functional.relu(self.conv1(x))
        out = self.fc(self.flatten(out))
        return out

# Dataset

In [230]:
class ArgoverseDataset(torch.utils.data.Dataset):
    """Dataset class for Argoverse"""
    
    def __init__(self, 
                 data_path,
                 sample_indices):
        super(ArgoverseDataset, self).__init__()
        
        self.data_path = data_path
        self.sample_indices = sample_indices
        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.sample_indices)

    def __getitem__(self, idx):
        
        # Load one scene
        pkl_path = self.pkl_list[self.sample_indices[idx]]
        with open(pkl_path, 'rb') as f:
            scene = pickle.load(f)
            
        # the index of agent to be predicted 
        pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
        
        # input: p_in & v_in; output: p_out
        p_in_raw = scene['p_in'][pred_id]
        p_out_raw = scene['p_out'][pred_id]
        v_in_raw = scene['v_in'][pred_id]
        lane_scene = scene['lane']
        
        # Normalization
        min_vecs = np.min(lane_scene, axis = 0)
        max_vecs = np.max(lane_scene, axis = 0)
        
        # Normalize by vectors
        p_in_normalized = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
        p_out_normalized = (p_out_raw - min_vecs)/(max_vecs - min_vecs)
        v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
        v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
        v_in_normalized = v_in_raw / v_in_norm
        inp = np.concatenate((p_in_normalized,v_in_normalized),axis=1)

        # Convert to float torch tensor
        return torch.from_numpy(inp).T.float(), torch.from_numpy(p_out_normalized).flatten().float()

In [231]:
class PVDataset(torch.utils.data.Dataset):
    def __init__(self, 
                 x_data_path,
                 y_data_path):
        super(PVDataset, self).__init__()
        
        self.X = np.load(x_data_path)
        self.y = np.load(y_data_path)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        sample_x = self.X[idx]
        sample_y = self.y[idx]
        return torch.from_numpy(sample_x).float(), torch.from_numpy(sample_y).float()

# Data Loader

In [232]:
# train_path = "./train"

# # total number of scenes
# indices = np.arange(0, 205942)

# # train-valid split
# np.random.shuffle(indices)
# train_indices = indices[:180000]
# valid_indices = indices[180000:]

# # define datasets
# train_set = PVDataset('train_X_cnn.npy', 'train_y_cnn.npy', train_indices)
# valid_set = PVDataset('train_X_cnn.npy', 'train_y_cnn.npy', valid_indices)

# # create dataloaders
# train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
# valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

# Model, Loss Function and Optimizer

In [233]:
model = CNNModel().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=decay_rate)

# Training

In [234]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def train_epoch(train_loader, model, optimizer, criterion):
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)

        loss = criterion(outputs, targets)
        loss.backward()

        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader)

def eval_epoch(valid_loader, model, criterion):
    val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in valid_loader:
            inputs = inputs.to(device)
            targets = targets.to(device)

            outputs = model(inputs)

            loss = criterion(outputs, targets)
            val_loss += loss.item() * inputs.size(0)

    return val_loss / len(valid_loader.dataset)

In [235]:
train_rmse = []
valid_rmse = []
min_rmse = 10e8

dataset = PVDataset('train_X_cnn.npy', 'train_y_cnn.npy')

for epoch in range(num_epoch):
    start = time.time()
    train_set, valid_set = random_split(
        dataset=dataset,
        lengths=[0.9, 0.1],
        generator=torch.Generator().manual_seed(0)
    )

    # create dataloaders
    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)
    valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, shuffle=False)

    model.train() # if you use dropout or batchnorm. 
    train_rmse.append(train_epoch(train_loader, model, optimizer, criterion))

    model.eval()
    valid_rmse.append(eval_epoch(valid_loader, model, criterion))

    # save the best model
    if valid_rmse[-1] < min_rmse:
        min_rmse = valid_rmse[-1] 
        best_model = model
        torch.save([best_model, epoch, get_lr(optimizer)], model_name)

    end = time.time()
    
    # Early Stopping
    if (len(train_rmse) > 100 and np.mean(valid_rmse[-5:]) >= np.mean(valid_rmse[-10:-5])):
            break       

    # Learning Rate Decay        
    scheduler.step()
    
    print("Epoch {} | T: {:0.2f} | Train RMSE: {:0.5f} | Valid RMSE: {:0.5f}".format(epoch + 1, (end-start) / 60, train_rmse[-1], valid_rmse[-1]))

Epoch 1 | T: 0.10 | Train RMSE: 1.85105 | Valid RMSE: 0.00298
Epoch 2 | T: 0.10 | Train RMSE: 0.41724 | Valid RMSE: 0.00222
Epoch 3 | T: 0.09 | Train RMSE: 0.37443 | Valid RMSE: 0.00139
Epoch 4 | T: 0.09 | Train RMSE: 0.34689 | Valid RMSE: 0.00139
Epoch 5 | T: 0.10 | Train RMSE: 0.34033 | Valid RMSE: 0.00199
Epoch 6 | T: 0.11 | Train RMSE: 0.32401 | Valid RMSE: 0.00296
Epoch 7 | T: 0.10 | Train RMSE: 0.31824 | Valid RMSE: 0.00130
Epoch 8 | T: 0.11 | Train RMSE: 0.31211 | Valid RMSE: 0.00136
Epoch 9 | T: 0.10 | Train RMSE: 0.30045 | Valid RMSE: 0.00150
Epoch 10 | T: 0.10 | Train RMSE: 0.29263 | Valid RMSE: 0.00134
Epoch 11 | T: 0.09 | Train RMSE: 0.28769 | Valid RMSE: 0.00117
Epoch 12 | T: 0.10 | Train RMSE: 0.28499 | Valid RMSE: 0.00106
Epoch 13 | T: 0.09 | Train RMSE: 0.27579 | Valid RMSE: 0.00225
Epoch 14 | T: 0.10 | Train RMSE: 0.27700 | Valid RMSE: 0.00111
Epoch 15 | T: 0.10 | Train RMSE: 0.26682 | Valid RMSE: 0.00165
Epoch 16 | T: 0.09 | Train RMSE: 0.26427 | Valid RMSE: 0.00170
E

# Evaluation and Submission

In [236]:
best_model, _, _ = torch.load(model_name)
best_model

CNNModel(
  (conv1): Conv1d(4, 32, kernel_size=(5,), stride=(1,))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Sequential(
    (0): Linear(in_features=480, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=60, bias=True)
  )
)

In [237]:
test_path = "./val_in/val_in/"
test_pkl_list = glob(os.path.join(test_path, '*'))
test_pkl_list.sort()

test_preds = []
best_model.eval()
for test_pkl_path in tqdm(test_pkl_list):
    with open(test_pkl_path, 'rb') as f:
        scene = pickle.load(f)
        pred_id = np.where(scene["track_id"] == scene['agent_id'])[0][0]
        # input: p_in & v_in; output: p_out
        p_in_raw = scene['p_in'][pred_id]
        v_in_raw = scene['v_in'][pred_id]
        lane_scene = scene['lane']
        
        # Normalization
        min_vecs = np.min(lane_scene, axis = 0)
        max_vecs = np.max(lane_scene, axis = 0)
        
        # Normalize by vectors
        p_in_normalized = (p_in_raw - min_vecs)/(max_vecs - min_vecs)
        v_in_norm = np.linalg.norm(v_in_raw, axis=1, keepdims=True)
        v_in_norm = np.where(v_in_norm == 0.0, 1.0, v_in_norm)
        v_in_normalized = v_in_raw / v_in_norm
        inp = np.concatenate((p_in_normalized,v_in_normalized),axis=1)
        inp = torch.from_numpy(inp).float().T.unsqueeze(0).to(device)

        output = best_model(inp)
        output = output.cpu().detach().numpy()
        output.resize((30,2))
        # De-Normalization ! 
        output = output * (max_vecs[:2] - min_vecs[:2]) +  min_vecs[:2]
        test_preds.append(output)

100%|██████████| 3200/3200 [00:04<00:00, 645.55it/s]


# Generate Submission File

In [238]:
# # Submission Files
sample_sub = pd.read_csv('sample_submission.csv')

In [239]:
# Convert to int
predictions = np.concatenate(test_preds).reshape(len(test_preds), -1).astype(int)
sub_df = pd.DataFrame(np.c_[sample_sub["ID"], predictions], columns=[np.r_[["ID"], ["v" + str(i) for i in range(1, 61)]]])
sub_df.to_csv(submission_name, index=None)
sub_df.dtypes

ID     int64
v1     int64
v2     int64
v3     int64
v4     int64
       ...  
v56    int64
v57    int64
v58    int64
v59    int64
v60    int64
Length: 61, dtype: object

In [240]:
sub_df.dtypes

ID     int64
v1     int64
v2     int64
v3     int64
v4     int64
       ...  
v56    int64
v57    int64
v58    int64
v59    int64
v60    int64
Length: 61, dtype: object